In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
tf.__version__


'1.2.1'

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("data/MNIST/", one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [5]:
data.test.labels[0:5, :]
data.test.cls = data.test.labels.argmax(axis=1)


In [6]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 28
 # Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size
 # Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)
 # Number of classes, one class for each of 10 digits.
num_classes = 10

In [7]:
x = tf.placeholder(tf.float32, [None, img_size_flat])
y_true = tf.placeholder(tf.float64, [None, num_classes])
y_true_cls = tf.placeholder(tf.int64, [None])

In [8]:
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500
REGULARAZTION_RATE = 0.0001
def get_weight_variable(shape, regularizer):
    weights = tf.get_variable("weights", shape,initializer=tf.truncated_normal_initializer(stddev=0.1)) #生成截断正态分布的随机数,标准差为0.1
    if regularizer != None:
        tf.add_to_collection('losses', regularizer(weights))
    return weights

In [9]:
#define the forward network
def inference(input_tensor, regularizer):
    with tf.variable_scope('layer1'):#声明第一层神经网络的变量并完成前向传播过程
        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)  #tf.nn.relu是作为激活函数

    with tf.variable_scope('layer2'):#声明第二层神经网络的变量并完成前向传播过程
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    return layer2

In [12]:
regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)  
logits = inference(x, None)


In [13]:
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [14]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_true)
cost = tf.reduce_mean(cross_entropy)


In [15]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cost)


In [16]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
session = tf.Session()
session.run(tf.global_variables_initializer())


In [18]:
batch_size = 100
def optimize(num_iterations):
    for i in range(num_iterations):
        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch = data.train.next_batch(batch_size)
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        # Note that the placeholder for y_true_cls is not set
        # because it is not used during training.
        feed_dict_train = {x: x_batch, y_true: y_true_batch}
        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)

In [19]:
feed_dict_test = {x: data.test.images,
                  y_true: data.test.labels,
                  y_true_cls: data.test.cls}

In [20]:
optimize(num_iterations=1)



In [21]:
def print_accuracy():
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed_dict_test)
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [22]:
print_accuracy()

Accuracy on test-set: 19.8%


In [23]:
optimize(num_iterations=1)
print_accuracy()



Accuracy on test-set: 26.8%


In [24]:
optimize(num_iterations=400)
print_accuracy()

Accuracy on test-set: 95.2%


In [25]:
optimize(num_iterations=900)
print_accuracy()

Accuracy on test-set: 97.0%
